In [1]:
import pandas as pd
import numpy as np
from HFC125_hmimFAP_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = -0.20
    init_kappa_1_2A = -0.016
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R125'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['hmimFAP'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R125'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['hmimFAP','hmimFAP'].fix(0)
    m.fs.properties.PR_kappa_A['hmimFAP', 'R125'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_A['R125', 'hmimFAP'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['hmimFAP', 'hmimFAP'].fix(0)
    m.fs.properties.PR_kappa_B['hmimFAP', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'hmimFAP'].fix(0)
    m.fs.properties.PR_kappa_C['hmimFAP', 'hmimFAP'].fix(0)
    m.fs.properties.PR_kappa_C['hmimFAP', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'hmimFAP'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['hmimFAP'].unfix()
    m.fs.state_block.mole_frac_comp['R125'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data['x_R125']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'hmimFAP'].fix(float(data['x_hmimFAP']))
    m.fs.state_block.mole_frac_comp['R125'].fix(float(data['x_R125'])+eps)
    m.fs.state_block.mole_frac_comp['hmimFAP'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['hmimFAP', 'R125'].setlb(-5)
    m.fs.properties.PR_kappa_A['hmimFAP', 'R125'].setub(5)

    m.fs.properties.PR_kappa_A['R125', 'hmimFAP'].setlb(-5)
    m.fs.properties.PR_kappa_A['R125', 'hmimFAP'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['hmimFAP', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'hmimFAP']"]


In [3]:
data = pd.read_csv('R125_hmimFAP.csv')

print(data)

   temperature  pressure  x_R125  x_hmimFAP
0       298.15     50000   0.038      0.962
1       298.15    100000   0.074      0.926
2       298.15    200000   0.142      0.858
3       298.15    400000   0.267      0.733
4       298.15    600000   0.380      0.620
5       298.15    800000   0.482      0.518
6       298.15   1000000   0.578      0.422


In [4]:
N = 100 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.10 #mole fraction

for i in range(N):
    print(i)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
    data_["x_hmimFAP"] += data_["x_hmimFAP"] * noise_x
    data_.loc[data_['x_hmimFAP'] <= 0.0001, 'x_hmimFAP'] = 0.0001
    data_.loc[data_['x_hmimFAP'] >= 0.9999 , 'x_hmimFAP'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
-0.04790429794807946
2022-03-06 12:13:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:13:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:13:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:13:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:13:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:13:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:13:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimFAP,R125] = -0.13625452050079642
fs.properties.PR_kappa_A[R125,hmimFAP] = -0.0445242181246251
1
-0.26

2022-03-06 12:14:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimFAP,R125] = -0.13412551861031255
fs.properties.PR_kappa_A[R125,hmimFAP] = -0.042372612179110335
8
-0.038016513770911205
2022-03-06 12:14:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:14:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:14:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:15:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:15:01 [INFO] idaes.init.fs.state_block: Pro

2022-03-06 12:16:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:16:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:16:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:16:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:16:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:16:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:16:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasibl

2022-03-06 12:17:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:17:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:17:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:17:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:17:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimFAP,R125] = -0.12692535777742514
fs.properties.PR_kappa_A[R125,hmimFAP] = -0.04305472306586375
23
0.0722763701042307
2022-03-06 12:17:19 [INFO] idaes.init.fs.state_block: Proper

2022-03-06 12:18:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:18:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:18:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:18:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:18:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimFAP,R125] = -0.14671048061137196
fs.properties.PR_kappa_A[R125,hmimFAP] = -0.049564758711753906
30
0.07762747790114852
2022-03-06 12:18:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:18:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-

...

2022-03-06 12:24:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:24:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:24:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:24:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:24:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimFAP,R125] = -0.13447431557691
fs.properties.PR_kappa_A[R125,hmimFAP] = -0.04230417301422014
67
0.05420266555197946
2022-03-06 12:24:13 [INFO] idaes.init.fs.state_block: Property

2022-03-06 12:25:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimFAP,R125] = -0.13198687199902828
fs.properties.PR_kappa_A[R125,hmimFAP] = -0.04256864411790718
74
0.042731839701664824
2022-03-06 12:25:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:25:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:25:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:25:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:25:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:25:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-

2022-03-06 12:26:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:26:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:26:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:26:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:26:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:26:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:26:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimFAP,R125] = -0.1327502022421084
fs.properties.PR_kappa_A[R125,hmimFAP] = -0.042914490869038126
82
0.0631422787953541
2022-03

2022-03-06 12:28:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:28:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:28:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimFAP,R125] = -0.14818410746498697
fs.properties.PR_kappa_A[R125,hmimFAP] = -0.05030687146342374
89
-0.10979294473587434
2022-03-06 12:28:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:28:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:28:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:28:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-

fs.properties.PR_kappa_A[hmimFAP,R125] = -0.1354368261230151
fs.properties.PR_kappa_A[R125,hmimFAP] = -0.041787541431439236
96
0.010316975620976762
2022-03-06 12:29:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:29:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:29:48 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:29:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:29:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:29:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:29:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    m

In [5]:
print(results_params)

[[-0.13625452 -0.04452422]
 [-0.17257383 -0.06405257]
 [-0.14367291 -0.048061  ]
 [-0.13531151 -0.04196432]
 [-0.12875164 -0.04112092]
 [-0.13472773 -0.04381835]
 [-0.13533237 -0.04160969]
 [-0.13412552 -0.04237261]
 [-0.13453736 -0.04373084]
 [-0.13536158 -0.04190312]
 [-0.1309171  -0.04208674]
 [-0.13140532 -0.04282682]
 [-0.13261258 -0.04285202]
 [-0.13342604 -0.04248787]
 [-0.13387309 -0.04342631]
 [-0.13449039 -0.04229798]
 [-0.12829399 -0.04298166]
 [-0.1341461  -0.04236971]
 [-0.13509012 -0.04206512]
 [-0.1343415  -0.04364091]
 [-0.13193503 -0.04254522]
 [-0.14391511 -0.04817965]
 [-0.12692536 -0.04305472]
 [-0.13454909 -0.04227536]
 [-0.13517247 -0.04203319]
 [-0.14295499 -0.04771052]
 [-0.14263187 -0.04755338]
 [-0.13372161 -0.04244073]
 [-0.13432236 -0.04234466]
 [-0.14671048 -0.04956476]
 [-0.13442742 -0.04231855]
 [-0.14181865 -0.04715951]
 [-0.13701784 -0.04487981]
 [-0.14249624 -0.04748752]
 [-0.13741836 -0.04506713]
 [-0.13888648 -0.04575812]
 [-0.13543088 -0.04168132]
 

In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R125_hmimFAP_10_2.csv')